In [1]:
import numpy as np
import pandas as pd
import polars as pl

# See readme for how I obtained the the below data
#catalog = pd.read_csv('../data/1680058493237O-result.csv', sep=',', header=0, dtype=np.float64)

# smaller sampe of the above data for testing purposes
catalog = pd.read_csv('../data/test.csv', sep=',', header=0, dtype=np.float64)

# arbitrary threashold 'k is for Kalle'
threshold_k = np.pi / 4

# helper functio
deg2rad = lambda rad: rad * np.pi / 180

# sort the catalog by magnitudes
stars = catalog.sort_values(by='b1mag')



In [2]:
# This bit of math is a common operation in astronomy and is used to convert the spherical coordinates (r, \theta, \phi) to cartesian coordinates (x, y, z)
# To be more specific, if (RA, Dec) represents the spherical coordinates of a celestial object, then we can convert them to Cartesian coordinates (x, y, z) 
# using the following formulas:
#   x = cos(dec) * cos(ra)
#   y = cos(dec) * sin(ra)
#   z = sin(dec)
#
rays = np.stack(
            (   np.cos(deg2rad(stars['dec'])) * np.cos(deg2rad(stars['ra'])), 
                np.cos(deg2rad(stars['dec'])) * np.sin(deg2rad(stars['ra'])), 
                np.sin(deg2rad(stars['dec']))
            ) 
        ).T
rays

# Conversely, if we have the Cartesian coordinates (x, y, z), we can convert them to spherical coordinates (r, θ, φ) using the following formulas:
#
# r = sqrt(x^2 + y^2 + z^2)
# θ = arccos(z/r)
# φ = atan2(y, x)


array([[-0.18745319,  0.93921852, -0.287628  ],
       [-0.06322262,  0.60274195, -0.79542759],
       [-0.37385775, -0.31259124, -0.87322225],
       [-0.78378669, -0.52698704,  0.32857736],
       [-0.04767171,  0.08311572,  0.99539901],
       [-0.11978572,  0.27115942,  0.95505181],
       [-0.2945663 ,  0.02230101,  0.9553708 ],
       [-0.05242118,  0.37907628,  0.92387943],
       [-0.24657199, -0.29256175,  0.9239101 ],
       [ 0.05017215, -0.41511951,  0.90838238],
       [ 0.06515065, -0.05145824,  0.99654776],
       [ 0.12900926,  0.25874114,  0.95729287],
       [-0.04268434, -0.28477824,  0.95764263]])

Now we have an array of cartesian coordinates tuples (x,y,z) for each star in the catalog.  Magnitude is ignored (for now). 

In [3]:
numrows = rays.shape[0]

#r1 = rays[0,:]  # get the row at position 0 of the rays collection
#display(r1)

#r2 = rays[0+1:,:] # gets the next row of the rays collection  
#display(r2)

#display(r1.sum(-1))

#rr = rays[0,:] * rays[0+1:,:] # gets the next row of the rays collection  
#display(rr)

#sum = (rays[0,:] * rays[0+1:,:]).sum(-1) # gets the next row of the rays collection  
#display("sum is ")
#display(sum)
#display("sum is ")
#display(rays[0+1:,:].sum(-1))



#rays[0+1:,:] # this is where I loose track?  
d = np.arccos((rays[0,:] * rays[0+1:,:]).sum(-1) )

#display(d)
flat = np.flatnonzero(d < threshold_k)
#display(flat)

for i in range(0, numrows):
    d = np.arccos((rays[i,:] * rays[i+1:,:]).sum(-1) )
    display(d)


array([0.63217378, 1.54314028, 2.02922629, 1.77144455, 1.5683638 ,
       1.77075465, 1.47049987, 2.08782701, 2.29237282, 1.92535686,
       1.62733941, 2.13520053])

array([1.01242041, 2.12874094, 2.40187153, 2.20019929, 2.38599709,
       2.09795502, 2.6806956 , 2.92176186, 2.5460008 , 2.23148319,
       2.76707597])

array([1.39911907, 2.64113565, 2.63407106, 2.39072365, 2.70370631,
       2.24355224, 2.32158312, 2.64346645, 2.87639517, 2.39096047])

array([1.24440272, 1.30280256, 1.00860915, 1.42539998, 0.86187938,
       1.07251993, 1.26243607, 1.49364401, 1.04928682])

array([0.20576182, 0.25812154, 0.30570547, 0.43446058, 0.52102751,
       0.17584067, 0.25268866, 0.37200122])

...

array([0.32283219, 0.40345279, 0.68099812, 0.20717613])

array([0.37671935, 0.69406496, 0.16764098])

array([0.3204977 , 0.26069959])

array([0.57800565])

array([], dtype=float64)

In [4]:
i_, j_, d_ = [],[],[]
for i in range(0, numrows):
    d = np.arccos((rays[i,:] * rays[i+1:,:]).sum(-1) )
    for j in np.flatnonzero(d < threshold_k):
        i_.append(i)
        j_.append(i+1+j)
        d_.append(d[j])

pairs = pd.DataFrame({'i':np.array(i_), 'j':np.array(j_), 'd':np.array(d_)}).sort_values(by='d')

pairs


,i,j,d
10,5,7,0.131074
32,9,12,0.167641
6,4,10,0.175841
1,4,5,0.205762
29,8,12,0.207176
24,7,11,0.220707
14,5,11,0.249763
7,4,11,0.252689
2,4,6,0.258122
34,10,12,0.260700
